## Inicializando o pyspark e criando o SparkContext

In [1]:
import pyspark 
from pyspark.sql import SQLContext
# to start a spark context

sc=SparkContext.getOrCreate()   
    
spark = SparkSession.builder.appName("TweetSentiApp").getOrCreate()

## Lendo o arquivo CSV

In [2]:
print("************************READING CSV*******************************")

df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('hdfs://localhost:9000/user/input/datasetReviewed.csv')
type(df)

print("************************CSV OK************************************")

************************READING CSV*******************************
************************CSV OK************************************


## Droppando algumas colunas que não serão necessárias

In [3]:
drop_list = ['company', 'company_count', 'created_at', 'favorite_count', 'retweet_count']
df = df.select([column for column in df.columns if column not in drop_list])

## Modificando valores por string via SQL  -- teste --

In [4]:
#df.createOrReplaceTempView("temp")
#df = spark.sql('SELECT CASE polaridade WHEN -1 THEN "negativo" WHEN 0 THEN "neutro" ELSE "positivo" END AS polaridade,tweet FROM temp')

#df.show()

## Mostrando o dataframe com as colunas selecionadas

In [5]:
df.show(truncate=20)

+--------------------+-------------+--------------------+----------+
|            tweet_id|  screen_name|               tweet|polaridade|
+--------------------+-------------+--------------------+----------+
| 1128984581330407424|AgenciaEstado|RT @colunadobroad...|         0|
| 1128600913092857856|AgenciaEstado|Iguatemi vai repe...|      null|
|https://t.co/ZTml...|         null|                null|      null|
| 1128245224294293504|AgenciaEstado|RT @colunadobroad...|         1|
| 1127542253159952384|AgenciaEstado|RT @colunadobroad...|         1|
| 1127153121363419137|AgenciaEstado|Iguatemi fecha pa...|         1|
| 1126927749791277059|AgenciaEstado|As três prioridad...|         1|
| 1126435268192559104|AgenciaEstado|Atacada, Cielo di...|         0|
| 1126078103917334529|AgenciaEstado|Fique de olho: Me...|         0|
| 1125715108217466880|AgenciaEstado|Fique de olho: Co...|         0|
| 1124275377000603649|AgenciaEstado|RT @colunadobroad...|         1|
| 1123883457438994432|AgenciaEstad

## Droppando linhas com NaN e contando quantas linhas o dataframe possui

In [6]:
df = df.dropna()
df.show()

+-------------------+-------------+--------------------+----------+
|           tweet_id|  screen_name|               tweet|polaridade|
+-------------------+-------------+--------------------+----------+
|1128984581330407424|AgenciaEstado|RT @colunadobroad...|         0|
|1128245224294293504|AgenciaEstado|RT @colunadobroad...|         1|
|1127542253159952384|AgenciaEstado|RT @colunadobroad...|         1|
|1127153121363419137|AgenciaEstado|Iguatemi fecha pa...|         1|
|1126927749791277059|AgenciaEstado|As três prioridad...|         1|
|1126435268192559104|AgenciaEstado|Atacada, Cielo di...|         0|
|1126078103917334529|AgenciaEstado|Fique de olho: Me...|         0|
|1125715108217466880|AgenciaEstado|Fique de olho: Co...|         0|
|1124275377000603649|AgenciaEstado|RT @colunadobroad...|         1|
|1123883457438994432|AgenciaEstado|RT @colunadobroad...|         1|
|1123166798428495872|AgenciaEstado|RT @colunadobroad...|         0|
|1121366959512346627|AgenciaEstado|Fique de olho

## Separando o dataframe em um dataframe de treino (train_set)  com 80% e um dataframe de teste (test_set) com 20%

In [7]:
(train_set, test_set) = df.randomSplit([0.8, 0.2], seed = 0)

## Processo de limpeza, tirando links e pontuações

In [8]:
import re
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import *

udf = UserDefinedFunction(lambda x: re.sub(r"http\S+", "", x).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','').replace('"','').replace('rt',''), StringType())


df_clean = df.select(*[udf(column).alias(column) for column in df.columns])

df_clean.show()

+-------------------+-------------+--------------------+----------+
|           tweet_id|  screen_name|               tweet|polaridade|
+-------------------+-------------+--------------------+----------+
|1128984581330407424|agenciaestado| @colunadobroad d...|         0|
|1128245224294293504|agenciaestado| @colunadobroad i...|         1|
|1127542253159952384|agenciaestado| @colunadobroad i...|         1|
|1127153121363419137|agenciaestado|iguatemi fecha pa...|         1|
|1126927749791277059|agenciaestado|as três prioridad...|         1|
|1126435268192559104|agenciaestado|atacada, cielo di...|         0|
|1126078103917334529|agenciaestado|fique de olho mer...|         0|
|1125715108217466880|agenciaestado|fique de olho com...|         0|
|1124275377000603649|agenciaestado| @colunadobroad t...|         1|
|1123883457438994432|agenciaestado| @colunadobroad s...|         1|
|1123166798428495872|agenciaestado| @colunadobroad c...|         0|
|1121366959512346627|agenciaestado|fique de olho

## Processo de tokenização das palavras

In [9]:
from pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(inputCol="tweet", outputCol="words")
df_token = tokenizer.transform(df_clean)

In [10]:
df_token.show(truncate=30)

+-------------------+-------------+------------------------------+----------+------------------------------+
|           tweet_id|  screen_name|                         tweet|polaridade|                         words|
+-------------------+-------------+------------------------------+----------+------------------------------+
|1128984581330407424|agenciaestado| @colunadobroad diretor afa...|         0|[, @colunadobroad, diretor,...|
|1128245224294293504|agenciaestado| @colunadobroad irb brasil ...|         1|[, @colunadobroad, irb, bra...|
|1127542253159952384|agenciaestado| @colunadobroad itaú uniban...|         1|[, @colunadobroad, itaú, un...|
|1127153121363419137|agenciaestado|iguatemi fecha parceria com...|         1|[iguatemi, fecha, parceria,...|
|1126927749791277059|agenciaestado|as três prioridades daqui e...|         1|[as, três, prioridades, daq...|
|1126435268192559104|agenciaestado|atacada, cielo diz que não ...|         0|[atacada,, cielo, diz, que,...|
|112607810391733452

## Processo de retirada das StopWords

In [11]:
from pyspark.ml.feature import StopWordsRemover

remover = StopWordsRemover(inputCol="words", outputCol="filtered")
df_remover = remover.transform(df_token)

In [12]:
df_remover.show(truncate=20)

+-------------------+-------------+--------------------+----------+--------------------+--------------------+
|           tweet_id|  screen_name|               tweet|polaridade|               words|            filtered|
+-------------------+-------------+--------------------+----------+--------------------+--------------------+
|1128984581330407424|agenciaestado| @colunadobroad d...|         0|[, @colunadobroad...|[, @colunadobroad...|
|1128245224294293504|agenciaestado| @colunadobroad i...|         1|[, @colunadobroad...|[, @colunadobroad...|
|1127542253159952384|agenciaestado| @colunadobroad i...|         1|[, @colunadobroad...|[, @colunadobroad...|
|1127153121363419137|agenciaestado|iguatemi fecha pa...|         1|[iguatemi, fecha,...|[iguatemi, fecha,...|
|1126927749791277059|agenciaestado|as três prioridad...|         1|[as, três, priori...|[três, prioridade...|
|1126435268192559104|agenciaestado|atacada, cielo di...|         0|[atacada,, cielo,...|[atacada,, cielo,...|
|112607810

## Processo de Stemming 

In [13]:
import nltk
from pyspark.sql.functions import udf
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer(language='portuguese')
stemmer_udf = udf(lambda tokens: [stemmer.stem(token) for token in tokens])
df_stemmed = df_remover.withColumn("words_stemmed", stemmer_udf("filtered"))
df_stemmed.select("words_stemmed").show(truncate= 80)

+--------------------------------------------------------------------------------+
|                                                                   words_stemmed|
+--------------------------------------------------------------------------------+
|      [, @colunadobroad, diretor, afast, bb, é, indic, par, o, patagôn, @estada]|
|[, @colunadobroad, irb, brasil, re, busc, increment, negóci, méxic, @colunado...|
|[, @colunadobroad, itaú, unibanc, apost, em, nov, model, de, agênc, rum, ao, ...|
|[iguatem, fech, parc, com, @ifood, e, acirr, concorrent, nas, entreg, via, sh...|
|[três, prioridad, daqu, em, diant, sã, segurança,, pesso, e, reparação,, diz,...|
|[atacada,, ciel, diz, que, nã, vai, vir, uma, nov, kodak, #maquininh, #ciel, ...|
|[fiqu, de, olho, merc, olha, petrobr, e, balanc, chinesa,, com, prevident, ra...|
|[fiqu, de, olho, comérci, dos, eua, e, chin, nutr, cautel, em, mei, à, esper,...|
|[, @colunadobroad, tim, deslanch, na, ofe, de, cham, de, voz, pel, red, 4g, @...|
|[, 

## Processo TDIDF Vectorizer

In [14]:
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.feature import StringIndexer

hashtf = HashingTF(numFeatures=2**16, inputCol="filtered", outputCol='tf')
idf = IDF(inputCol='tf', outputCol="features", minDocFreq=3) 
label_stringIdx = StringIndexer(inputCol = "polaridade", outputCol = "label")

## Pipeline

In [15]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[tokenizer, remover, hashtf, idf, label_stringIdx,])
pipelineFit = pipeline.fit(train_set)
train_df = pipelineFit.transform(train_set)
test_df = pipelineFit.transform(test_set)
train_df.select("polaridade", "words", "filtered", "tf", "features", "label").show(truncate=20)

+----------+--------------------+--------------------+--------------------+--------------------+-----+
|polaridade|               words|            filtered|                  tf|            features|label|
+----------+--------------------+--------------------+--------------------+--------------------+-----+
|         1|[investidor, estr...|[investidor, estr...|(65536,[8618,9935...|(65536,[8618,9935...|  1.0|
|         1|[santander, e, we...|[santander, e, we...|(65536,[6399,1722...|(65536,[6399,1722...|  1.0|
|         0|[pedro, parente, ...|[pedro, parente, ...|(65536,[6160,1188...|(65536,[6160,1188...|  0.0|
|         1|[bb, se, une, à, ...|[bb, se, une, à, ...|(65536,[6568,1611...|(65536,[6568,1611...|  1.0|
|         1|[cubo, itaú, firm...|[cubo, itaú, firm...|(65536,[8717,1343...|(65536,[8717,1343...|  1.0|
|         0|[governo, envia, ...|[governo, envia, ...|(65536,[4119,9346...|(65536,[4119,9346...|  0.0|
|         0|[bônus, da, petro...|[bônus, da, petro...|(65536,[1715,9181..

In [16]:
train_df.select("features").show(truncate=80)

+--------------------------------------------------------------------------------+
|                                                                        features|
+--------------------------------------------------------------------------------+
|(65536,[8618,9935,19384,31870,35403,37144,46615,52354,53391,58033,60875,64607...|
|(65536,[6399,17222,18462,19208,20643,20848,29126,30818,41380,41399,49649,5257...|
|(65536,[6160,11887,18466,24833,25775,28851,34218,36449,41380,44461,46257,4989...|
|(65536,[6568,16117,29126,35373,36449,38227,40365,40729,41380,44212,50583,5257...|
|(65536,[8717,13439,20046,22873,23314,31488,37179,37955,38426,41380,48526,5257...|
|(65536,[4119,9346,15165,18750,20141,30535,35183,36449,37902,43101,45238,46257...|
|(65536,[1715,9181,14210,16117,38869,41380,44569,46257,52572,54518,60235,65422...|
|(65536,[1349,8872,11887,14804,18466,34658,36449,41380,42491,50218,52572,56916...|
|(65536,[8872,11858,17222,18302,18466,19013,21851,23666,24833,28297,36449,4138...|
|(65

In [17]:
train_df.count()

2135

## Processo de classificação de Naive Bayes no dataframe de teste

In [18]:
from pyspark.ml.classification import NaiveBayes

nb = NaiveBayes(smoothing=1, modelType="multinomial")
model = nb.fit(train_df)

predictions = model.transform(test_df) 

predictions.printSchema()

predictions.select("polaridade","filtered", "features", "label", "rawPrediction", "probability", "prediction") \
    .show(n = 20, truncate = 20)

root
 |-- tweet_id: string (nullable = true)
 |-- screen_name: string (nullable = true)
 |-- tweet: string (nullable = true)
 |-- polaridade: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- filtered: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- tf: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- label: double (nullable = false)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)

+----------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|polaridade|            filtered|            features|label|       rawPrediction|         probability|prediction|
+----------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|         0|[visa, e, cielo, ...|(65536,[3021,7729...|  0.0|[-95.32967414237

## Calculando a precisão da classificação no dataframe de teste

In [36]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
evaluator.evaluate(predictions)
#evaluator.explainParam("metricName")

0.7842401500938087

## ----------- test -----------

In [20]:
from pyspark.mllib.evaluation import MulticlassMetrics
# Create (prediction, label) pairs
predictionAndLabel = predictions.select("prediction", "label").rdd

# Generate confusion matrix
metrics = MulticlassMetrics(predictionAndLabel)

metrics.confusionMatrix()

DenseMatrix(3, 3, [239.0, 35.0, 16.0, 33.0, 157.0, 26.0, 3.0, 2.0, 22.0], 0)

In [37]:
predictions.groupBy('polaridade','prediction').count().show()

+----------+----------+-----+
|polaridade|prediction|count|
+----------+----------+-----+
|         1|       0.0|   35|
|         0|       2.0|    3|
|        -1|       1.0|   26|
|         0|       0.0|  239|
|         0|       1.0|   33|
|         1|       2.0|    2|
|         1|       1.0|  157|
|        -1|       0.0|   16|
|        -1|       2.0|   22|
+----------+----------+-----+



In [49]:
predictions.createOrReplaceTempView("temp")
predictions = spark.sql('SELECT * FROM temp WHERE prediction == 1 OR prediction == 0')

predictions.groupBy('polaridade','prediction').count().show()

+----------+----------+-----+
|polaridade|prediction|count|
+----------+----------+-----+
|         1|       0.0|   35|
|        -1|       1.0|   26|
|         0|       0.0|  239|
|         0|       1.0|   33|
|         1|       1.0|  157|
|        -1|       0.0|   16|
+----------+----------+-----+



## ----------- test -----------

## Passando o dataframe final para um arquivo CSV


In [51]:
predictions.toPandas().to_csv('datasetReviewed-testdataframe.csv', index_label="id-col")

## Processo de classificação de Naive Bayes com sklearn

In [23]:
#https://www.kaggle.com/leandrodoze/sentiment-analysis-in-portuguese/notebook

import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict

dataset = pd.read_csv('datasetReviewed.csv',encoding='utf-8')
dataset = dataset.dropna()
dataset.count()

tweet_id          2830
company           2830
company_count     2830
created_at        2830
favorite_count    2830
retweet_count     2830
screen_name       2830
tweet             2830
polaridade        2830
dtype: int64

In [24]:
tweets = dataset["tweet"].values
#tweets = pandas_df["filtered"].values

tweets

array(['RT @colunadobroad: Diretor afastado do BB é indicado para o Patagônia @estadao:  https://t.co/Lb1D94DIPE https://t.co/tlms2WZRG6',
       'Iguatemi vai repensar mix de lojistas para reverter queda na ocupação dos shoppings\nhttps://t.co/ZTmlbuFU8r Foto: Gabriela Biló/Estadão https://t.co/35VEeDYjRH',
       'RT @colunadobroad: IRB Brasil Re busca incrementar negócios no México @colunadobroad:  https://t.co/BzCXlaCcBJ https://t.co/BcfR48AicM',
       ...,
       'Mal-estar entre Casino e Klein pode atrasar processo de venda da Via Varejo -  https://t.co/OlpJtBSDBQ',
       'Controladores da Usiminas estão mais distantes de acordo -  https://t.co/aK8xynwiZq',
       'Patrice Etlin, da Advent, conversa com Michael Klein sobre Via Varejo -  https://t.co/2qjZua7V7H'],
      dtype=object)

In [25]:
classes = dataset["polaridade"].values
#classes = pandas_df["polaridade"].values

classes

array([0, 1, 1, ..., 0, 0, 0])

In [26]:
from nltk.corpus import stopwords

def Preprocessing(instancia):
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','').replace('"','').replace('rt','')
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    palavras = [i for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

tweets = [Preprocessing(i) for i in tweets]

tweets[:10]

['@colunadobroad diretor afastado bb indicado patagônia @estadao',
 'iguatemi vai repensar mix lojistas reveer queda ocupação shoppings foto gabriela biló/estadão',
 '@colunadobroad irb brasil re busca incrementar negócios méxico @colunadobroad',
 '@colunadobroad itaú unibanco aposta novo modelo agências rumo banco futuro @estadao',
 'iguatemi fecha parceria @ifood acirra concorrência entregas via shoppings foto helvio romero/estadão',
 'três prioridades daqui diante segurança, pessoas reparação, diz presidente interino vale foto wilton júnior/estadão',
 'atacada, cielo diz vai virar nova kodak #maquininhas #cielo @agenciaestado',
 'fique olho mercado olha petrobras balança chinesa, previdência radar antes copom',
 'fique olho comércio eua china nutre cautela meio espera previdência petrobras',
 '@colunadobroad tim deslancha ofea chamadas voz rede 4g @estadao']

In [27]:
from nltk.tokenize import TweetTokenizer
tweet_tokenizer = TweetTokenizer() 

In [28]:
vectorizer = CountVectorizer(analyzer="word",  tokenizer=tweet_tokenizer.tokenize)
#vectorizer = CountVectorizer(ngram_range = (1, 2))

freq_tweets = vectorizer.fit_transform(tweets)

freq_tweets.shape

(2830, 4559)

In [29]:
modelo = MultinomialNB()
modelo.fit(freq_tweets, classes)
modelo

MultinomialNB()

In [30]:
resultados = cross_val_predict(modelo, freq_tweets, classes, cv = 3)
resultados

array([ 0, -1,  0, ...,  1, -1,  0])

In [31]:
metrics.accuracy_score(classes, resultados)

0.7187279151943463

In [32]:
print(pd.crosstab(classes, resultados, rownames = ["Real"], colnames = ["Predito"], margins = True))

Predito   -1     0     1   All
Real                          
-1       236    27   133   396
0        158   977   310  1445
1         78    90   821   989
All      472  1094  1264  2830


In [33]:
data_frame = pd.DataFrame(data = { "id": dataset["tweet_id"], "texto": dataset["tweet"],"polaridade": dataset["polaridade"], "sentimento": resultados })
data_frame

,id,texto,polaridade,sentimento
0,1128984581330407424,RT @colunadobroad: Diretor afastado do BB é in...,0,0
1,1128600913092857856,Iguatemi vai repensar mix de lojistas para rev...,1,-1
2,1128245224294293504,RT @colunadobroad: IRB Brasil Re busca increme...,1,0
3,1127542253159952384,RT @colunadobroad: Itaú Unibanco aposta em nov...,1,1
4,1127153121363419137,Iguatemi fecha parceria com @iFood e acirra co...,1,1
...,...,...,...,...
2825,846478513842651136,"CNF elege Sergio Rial, do Santander, como novo...",0,0
2826,846011317156827137,"GPA mantém desejo, mas venda da via varejo atr...",0,1
2827,845265994608193536,Mal-estar entre Casino e Klein pode atrasar pr...,0,1
2828,844949553161932803,Controladores da Usiminas estão mais distantes...,0,-1


In [34]:
data_frame.to_csv("tweets_classificados.csv", index = True)